In [50]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from MongoCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "aacUser1"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GSLogo.png' # Grazioso Salvere logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Jake Bayer SNHU CS-340 Dashboard'))),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water', 'value': 'wtr' },
            {'label': 'Mountain or Wilderness', 'value': 'mtwd'},
            {'label': 'Disaster or Individual Tracking', 'value': 'dit'},
            {'label': 'No Filter', 'value': 'nof'}
        ],
        value='nof'
    ),  
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable = "multi",
        column_selectable = "single",
        selected_columns = [],
        page_action = "native",
        page_size = 10,
        page_current = 0
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if(value == 'wtr'):
        df = pd.DataFrame.from_records(shelter.read({
            'animal_type': 'Dog', 
            'breed': {'$in': ['Labrador Retriever Mix', 
                      'Chesapeake Bay Retriever',
                      'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome': {'$in':['6 months', 
                                 '7 months',
                                 '8 months',
                                 '9 months',
                                 '10 months',
                                 '11 months',
                                 '1 year',
                                 '2 years',
                                 '3 years']}
            }))
        
    elif(value == 'mtwd'):
        df = pd.DataFrame.from_records(shelter.read({
            'animal_type': 'Dog', 
            'breed': {'$in':['German Shepherd', 
                      'Alaskan Malamute',
                      'Old English Sheepdog',
                      'Siberian Husky',
                      'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome': {'$in':['6 months', 
                                 '7 months',
                                 '8 months',
                                 '9 months',
                                 '10 months',
                                 '11 months',
                                 '1 year',
                                 '2 years',
                                 '3 years']}
        }))
        
    elif(value == 'dit'):
        df = pd.DataFrame.from_records(shelter.read({
            'animal_type': 'Dog', 
            'breed': {'$in': ['Doberman Pinscher',
                      'German Shepherd', 
                      'Golden Retriever',
                      'Bloodhound',
                      'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome': {'$in': ['5 months',
                                 '6 months', 
                                 '7 months',
                                 '8 months',
                                 '9 months',
                                 '10 months',
                                 '11 months',
                                 '1 year',
                                 '2 years',
                                 '3 years',
                                 '4 years',
                                 '5 years', 
                                 '6 years']}
        }))
        
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(data):
    dff = pd.DataFrame.from_dict(data)
    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center = [30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"), 
            # Marker with tooltip and popup
            dl.Marker(position =[30.75, -97.48], children=[
                dl.Tooltip(dff.iloc[0,3]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,8])
                ])
            ])
        ])
    ]



app